In [ ]:
from tensorrt_llm import LLM, SamplingParams


prompts = [

    "Hello, my name is",

    "The president of the United States is",

    "The capital of France is",

    "The future of AI is",

]

sampling_params = SamplingParams(temperature=0.8, top_p=0.95)


llm = LLM(model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")


outputs = llm.generate(prompts, sampling_params)


# Print the outputs.

for output in outputs:

    prompt = output.prompt

    generated_text = output.outputs[0].text

    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

In [ ]:
!python -c "import tensorrt_llm; print(tensorrt_llm._utils.trt_version())"

In [ ]:
!python -c "import sys; print(sys.executable)"

In [ ]:
!pip install --no-cache-dir --extra-index-url https://pypi.nvidia.com tensorrt_bindings==9.2.0.post12.dev5

In [ ]:
import tensorrt as trt
trt.__version__
# '9.3.0.post12.dev1'

In [2]:
!python -c "import tensorrt_llm; print(tensorrt_llm._utils.trt_version())"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "c:\Users\medha\OneDrive\Desktop\WorkSpace\tensorrt_llm\TensorRT-LLM-main\TensorRT-LLM-main\tensorrt_llm\__init__.py", line 32, in <module>
    import tensorrt_llm.functional as functional
  File "c:\Users\medha\OneDrive\Desktop\WorkSpace\tensorrt_llm\TensorRT-LLM-main\TensorRT-LLM-main\tensorrt_llm\functional.py", line 28, in <module>
    from . import graph_rewriting as gw
  File "c:\Users\medha\OneDrive\Desktop\WorkSpace\tensorrt_llm\TensorRT-LLM-main\TensorRT-LLM-main\tensorrt_llm\graph_rewriting.py", line 12, in <module>
    from .network import Network
  File "c:\Users\medha\OneDrive\Desktop\WorkSpace\tensorrt_llm\TensorRT-LLM-main\TensorRT-LLM-main\tensorrt_llm\network.py", line 27, in <module>
    from tensorrt_llm.module import Module
  File "c:\Users\medha\OneDrive\Desktop\WorkSpace\tensorrt_llm\TensorRT-LLM-main\TensorRT-LLM-main\tensorrt_llm\module.py", line 17, in <module>
    from ._common im

In [1]:
import tensorrt as trt
trt.__version__

'9.3.0.post12.dev1'

In [6]:
import torch
import torchvision.models as models

# Load a pre-trained model
model = models.resnet50(pretrained=True)
model.eval()

# Create a dummy input
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
torch.onnx.export(model, dummy_input, "resnet50.onnx")


C:\Users\medha\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\medha\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

# Load the ONNX model
with open("resnet50.onnx", "rb") as model:
    onnx_model = model.read()

# Create a TensorRT logger
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

# Create a TensorRT builder and network
builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

# Create a builder config
config = builder.create_builder_config()
config.max_workspace_size = 1 << 30  # 1 GB

# Parse the ONNX model
parser = trt.OnnxParser(network, TRT_LOGGER)
if not parser.parse(onnx_model):
    for error in range(parser.num_errors):
        print(parser.get_error(error))

# Build the TensorRT engine
engine = builder.build_engine(network, config)

# Save the engine to a file
with open("resnet50.trt", "wb") as f:
    f.write(engine.serialize())


C:\Users\medha\AppData\Local\Temp\ipykernel_8708\1146234682.py:18: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = 1 << 30  # 1 GB
C:\Users\medha\AppData\Local\Temp\ipykernel_8708\1146234682.py:27: DeprecationWarning: Use build_serialized_network instead.
  engine = builder.build_engine(network, config)


In [14]:
import torch
import torchvision.models as models
import onnx
import onnxruntime
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import time

# Load a pre-trained ResNet model from PyTorch
model = models.resnet50(pretrained=True)
model.eval()

# Create a dummy input
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
torch.onnx.export(model, dummy_input, "resnet50.onnx")

# Load the ONNX model
onnx_model = onnx.load("resnet50.onnx")
onnx.checker.check_model(onnx_model)

# Create a TensorRT logger
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

# Create a TensorRT builder and network
builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

# Create a builder config
config = builder.create_builder_config()
config.max_workspace_size = 1 << 30  # 1 GB

# Parse the ONNX model
parser = trt.OnnxParser(network, TRT_LOGGER)
if not parser.parse(onnx_model.SerializeToString()):
    for error in range(parser.num_errors):
        print(parser.get_error(error))

# Build the TensorRT engine
engine = builder.build_engine(network, config)

# Save the engine to a file
with open("resnet50.trt", "wb") as f:
    f.write(engine.serialize())

# Function to run inference with PyTorch model
def run_pytorch_inference(model, input_data):
    start_time = time.time()
    with torch.no_grad():
        output = model(input_data)
    end_time = time.time()
    return output, end_time - start_time

# Function to run inference with TensorRT engine
def run_tensorrt_inference(engine, input_data):
    context = engine.create_execution_context()

    input_shape = (1, 3, 224, 224)
    output_shape = (1, 1000)
    d_input = cuda.mem_alloc(trt.volume(input_shape) * np.dtype(np.float32).itemsize)
    d_output = cuda.mem_alloc(trt.volume(output_shape) * np.dtype(np.float32).itemsize)

    stream = cuda.Stream()

    cuda.memcpy_htod_async(d_input, input_data.numpy(), stream)

    start_time = time.time()
    context.execute_async_v2(bindings=[int(d_input), int(d_output)], stream_handle=stream.handle)
    stream.synchronize()
    end_time = time.time()

    output_data = np.empty(output_shape, dtype=np.float32)
    cuda.memcpy_dtoh_async(output_data, d_output, stream)
    stream.synchronize()

    return output_data, end_time - start_time

# Run inference with PyTorch model
pytorch_output, pytorch_time = run_pytorch_inference(model, dummy_input)

# Load the TensorRT engine
with open("resnet50.trt", "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
    engine = runtime.deserialize_cuda_engine(f.read())

# Run inference with TensorRT engine
tensorrt_output, tensorrt_time = run_tensorrt_inference(engine, dummy_input)

# Print the results
print(f"PyTorch Inference Time: {pytorch_time:.4f} seconds")
print(f"TensorRT Inference Time: {tensorrt_time:.4f} seconds")

# Compare the outputs (optional)
print("Output comparison (first 10 values):")
print("PyTorch:", pytorch_output[0][:10])
print("TensorRT:", tensorrt_output[0][:10])


C:\Users\medha\AppData\Local\Temp\ipykernel_8708\1212506721.py:34: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = 1 << 30  # 1 GB
C:\Users\medha\AppData\Local\Temp\ipykernel_8708\1212506721.py:43: DeprecationWarning: Use build_serialized_network instead.
  engine = builder.build_engine(network, config)


PyTorch Inference Time: 0.1498 seconds
TensorRT Inference Time: 0.0010 seconds
Output comparison (first 10 values):
PyTorch: tensor([-0.5746, -0.0726, -0.1359, -0.7958,  0.3136, -1.3083, -0.1524, -0.6186,
         0.2323, -1.4385])
TensorRT: [-0.57496476 -0.07318939 -0.1347372  -0.794315    0.31442487 -1.3087459
 -0.1512366  -0.61902785  0.23159608 -1.4398581 ]


# mobilenet arch

In [1]:
import torch
import torchvision.models as models
import onnx
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import time

# Load a pre-trained MobileNetV2 model from PyTorch
model = models.mobilenet_v2(pretrained=True)
model.eval()

# Create a dummy input
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
torch.onnx.export(model, dummy_input, "mobilenet_v2.onnx")

# Load the ONNX model
onnx_model = onnx.load("mobilenet_v2.onnx")
onnx.checker.check_model(onnx_model)

# Create a TensorRT logger
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

# Create a TensorRT builder and network
builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

# Create a builder config
config = builder.create_builder_config()
config.max_workspace_size = 1 << 30  # 1 GB

# Parse the ONNX model
parser = trt.OnnxParser(network, TRT_LOGGER)
if not parser.parse(onnx_model.SerializeToString()):
    for error in range(parser.num_errors):
        print(parser.get_error(error))

# Build the TensorRT engine
engine = builder.build_engine(network, config)

# Save the engine to a file
with open("mobilenet_v2.trt", "wb") as f:
    f.write(engine.serialize())

# Function to run inference with PyTorch model
def run_pytorch_inference(model, input_data):
    start_time = time.time()
    with torch.no_grad():
        output = model(input_data)
    end_time = time.time()
    return output, end_time - start_time

# Function to run inference with TensorRT engine
def run_tensorrt_inference(engine, input_data):
    context = engine.create_execution_context()

    input_shape = (1, 3, 224, 224)
    output_shape = (1, 1000)
    d_input = cuda.mem_alloc(trt.volume(input_shape) * np.dtype(np.float32).itemsize)
    d_output = cuda.mem_alloc(trt.volume(output_shape) * np.dtype(np.float32).itemsize)

    stream = cuda.Stream()

    cuda.memcpy_htod_async(d_input, input_data.numpy(), stream)

    start_time = time.time()
    context.execute_async_v2(bindings=[int(d_input), int(d_output)], stream_handle=stream.handle)
    stream.synchronize()
    end_time = time.time()

    output_data = np.empty(output_shape, dtype=np.float32)
    cuda.memcpy_dtoh_async(output_data, d_output, stream)
    stream.synchronize()

    return output_data, end_time - start_time

# Run inference with PyTorch model
pytorch_output, pytorch_time = run_pytorch_inference(model, dummy_input)

# Load the TensorRT engine
with open("mobilenet_v2.trt", "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
    engine = runtime.deserialize_cuda_engine(f.read())

# Run inference with TensorRT engine
tensorrt_output, tensorrt_time = run_tensorrt_inference(engine, dummy_input)

# Print the results
print(f"PyTorch Inference Time: {pytorch_time:.4f} seconds")
print(f"TensorRT Inference Time: {tensorrt_time:.4f} seconds")

# Compare the outputs (optional)
print("Output comparison (first 10 values):")
print("PyTorch:", pytorch_output[0][:10])
print("TensorRT:", tensorrt_output[0][:10])


C:\Users\medha\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\medha\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\medha\AppData\Local\Temp\ipykernel_15744\234049197.py:33: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = 1 << 30  # 1 GB
C:\Users\medha\AppData\Local\Temp\ipykernel_15744\234049197.py:42: DeprecationWarning: Use build_serialized_network instead.
  engine = builder.build_engine(

PyTorch Inference Time: 0.0238 seconds
TensorRT Inference Time: 0.0000 seconds
Output comparison (first 10 values):
PyTorch: tensor([-1.8675,  0.5877,  1.1875,  0.9224,  1.8973,  3.7545,  2.6415, -0.4664,
        -0.5275, -1.7199])
TensorRT: [-1.8693478   0.5870544   1.1914711   0.9210633   1.9046966   3.754747
  2.6366646  -0.46833465 -0.5303647  -1.7165513 ]


In [5]:
import torch
import torchvision.models as models
import onnx
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import time

# Load a pre-trained MobileNetV2 model from PyTorch
model = models.mobilenet_v2(pretrained=True)
model.eval()

# Create a dummy input
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
torch.onnx.export(model, dummy_input, "mobilenet_v2.onnx")

# Load the ONNX model
onnx_model = onnx.load("mobilenet_v2.onnx")
onnx.checker.check_model(onnx_model)

# Create a TensorRT logger
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

# Create a TensorRT builder and network
builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

# Create a builder config
config = builder.create_builder_config()
config.max_workspace_size = 1 << 30  # 1 GB

# Parse the ONNX model
parser = trt.OnnxParser(network, TRT_LOGGER)
if not parser.parse(onnx_model.SerializeToString()):
    for error in range(parser.num_errors):
        print(parser.get_error(error))

# Build the TensorRT engine
engine = builder.build_engine(network, config)

# Save the engine to a file
with open("mobilenet_v2.trt", "wb") as f:
    f.write(engine.serialize())

# Function to run inference with PyTorch model
def run_pytorch_inference(model, input_data):
    start_time = time.time()
    with torch.no_grad():
        output = model(input_data)
    end_time = time.time()
    return output, end_time - start_time

# Function to run inference with TensorRT engine
def run_tensorrt_inference(engine, input_data):
    context = engine.create_execution_context()

    input_shape = (1, 3, 224, 224)
    output_shape = (1, 1000)
    d_input = cuda.mem_alloc(trt.volume(input_shape) * np.dtype(np.float32).itemsize)
    d_output = cuda.mem_alloc(trt.volume(output_shape) * np.dtype(np.float32).itemsize)

    stream = cuda.Stream()

    cuda.memcpy_htod_async(d_input, input_data.numpy(), stream)

    start_time = time.time()
    context.execute_async_v2(bindings=[int(d_input), int(d_output)], stream_handle=stream.handle)
    stream.synchronize()
    end_time = time.time()

    output_data = np.empty(output_shape, dtype=np.float32)
    cuda.memcpy_dtoh_async(output_data, d_output, stream)
    stream.synchronize()

    return output_data, end_time - start_time

# Run inference for 10 images with PyTorch model
pytorch_times = []
for _ in range(10):
    pytorch_output, pytorch_time = run_pytorch_inference(model, dummy_input)
    pytorch_times.append(pytorch_time)

# Load the TensorRT engine
with open("mobilenet_v2.trt", "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
    engine = runtime.deserialize_cuda_engine(f.read())

# Run inference for 10 images with TensorRT engine
tensorrt_times = []
for _ in range(10000):
    tensorrt_output, tensorrt_time = run_tensorrt_inference(engine, dummy_input)
    tensorrt_times.append(tensorrt_time)

# Calculate the mean inference times
pytorch_mean_time = np.mean(pytorch_times)
tensorrt_mean_time = np.mean(tensorrt_times)

# Print the results
print(f"PyTorch Mean Inference Time: {pytorch_mean_time:.4f} seconds")
print(f"TensorRT Mean Inference Time: {tensorrt_mean_time:.4f} seconds")

# Compare the outputs (optional)
print("Output comparison (first 10 values):")
print("PyTorch:", pytorch_output[0][:10])
print("TensorRT:", tensorrt_output[0][:10])


C:\Users\medha\AppData\Local\Temp\ipykernel_22660\335943676.py:33: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = 1 << 30  # 1 GB
C:\Users\medha\AppData\Local\Temp\ipykernel_22660\335943676.py:42: DeprecationWarning: Use build_serialized_network instead.
  engine = builder.build_engine(network, config)


PyTorch Mean Inference Time: 0.0193 seconds
TensorRT Mean Inference Time: 0.0011 seconds
Output comparison (first 10 values):
PyTorch: tensor([-1.8003,  0.8952,  1.5019,  1.0823,  2.4900,  4.0107,  2.8733, -0.6329,
        -0.8625, -1.3930])
TensorRT: [-1.8063918  0.8972742  1.5052394  1.0819196  2.4986641  4.008527
  2.8675916 -0.6365201 -0.867194  -1.388763 ]


In [2]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import onnx
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import time

# Load a pre-trained MobileNetV2 model from PyTorch
model = models.mobilenet_v2(pretrained=True)
model.eval()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder, filename)
            image = Image.open(img_path).convert('RGB')
            image = transform(image).unsqueeze(0)  # Add batch dimension
            images.append(image)
    return images

# Load images from the specified folder
image_folder = "C:\\Users\\medha\\OneDrive\\Desktop\\WorkSpace\\tensorrt_llm\\TensorRT-LLM-main\\TensorRT-LLM-main\\data"
images = load_images_from_folder(image_folder)

# Create a dummy input for ONNX export
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
torch.onnx.export(model, dummy_input, "mobilenet_v2.onnx")

# Load the ONNX model
onnx_model = onnx.load("mobilenet_v2.onnx")
onnx.checker.check_model(onnx_model)

# Create a TensorRT logger
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

# Create a TensorRT builder and network
builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

# Create a builder config
config = builder.create_builder_config()
config.max_workspace_size = 1 << 30  # 1 GB

# Parse the ONNX model
parser = trt.OnnxParser(network, TRT_LOGGER)
if not parser.parse(onnx_model.SerializeToString()):
    for error in range(parser.num_errors):
        print(parser.get_error(error))

# Build the TensorRT engine
engine = builder.build_engine(network, config)

# Save the engine to a file
with open("mobilenet_v2.trt", "wb") as f:
    f.write(engine.serialize())

# Function to run inference with PyTorch model
def run_pytorch_inference(model, input_data):
    start_time = time.time()
    with torch.no_grad():
        output = model(input_data)
    end_time = time.time()
    return output, end_time - start_time

# Function to run inference with TensorRT engine
def run_tensorrt_inference(engine, input_data):
    context = engine.create_execution_context()

    input_shape = (1, 3, 224, 224)
    output_shape = (1, 1000)
    d_input = cuda.mem_alloc(trt.volume(input_shape) * np.dtype(np.float32).itemsize)
    d_output = cuda.mem_alloc(trt.volume(output_shape) * np.dtype(np.float32).itemsize)

    stream = cuda.Stream()

    cuda.memcpy_htod_async(d_input, input_data.numpy(), stream)

    start_time = time.time()
    context.execute_async_v2(bindings=[int(d_input), int(d_output)], stream_handle=stream.handle)
    stream.synchronize()
    end_time = time.time()

    output_data = np.empty(output_shape, dtype=np.float32)
    cuda.memcpy_dtoh_async(output_data, d_output, stream)
    stream.synchronize()

    return output_data, end_time - start_time

# Run inference for all images with PyTorch model
pytorch_times = []
for image in images:
    pytorch_output, pytorch_time = run_pytorch_inference(model, image)
    pytorch_times.append(pytorch_time)

# Load the TensorRT engine
with open("mobilenet_v2.trt", "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
    engine = runtime.deserialize_cuda_engine(f.read())

# Run inference for all images with TensorRT engine
tensorrt_times = []
for image in images:
    tensorrt_output, tensorrt_time = run_tensorrt_inference(engine, image)
    tensorrt_times.append(tensorrt_time)

# Calculate the mean inference times
pytorch_mean_time = np.mean(pytorch_times)
tensorrt_mean_time = np.mean(tensorrt_times)

# Print the results
print(f"PyTorch Mean Inference Time: {pytorch_mean_time:.4f} seconds")
print(f"TensorRT Mean Inference Time: {tensorrt_mean_time:.4f} seconds")

# Compare the outputs (optional)
print("Output comparison (first 10 values):")
print("PyTorch:", pytorch_output[0][:10])
print("TensorRT:", tensorrt_output[0][:10])


C:\Users\medha\AppData\Local\Temp\ipykernel_22660\4245462046.py:59: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = 1 << 30  # 1 GB
C:\Users\medha\AppData\Local\Temp\ipykernel_22660\4245462046.py:68: DeprecationWarning: Use build_serialized_network instead.
  engine = builder.build_engine(network, config)


PyTorch Mean Inference Time: 0.0181 seconds
TensorRT Mean Inference Time: 0.0010 seconds
Output comparison (first 10 values):
PyTorch: tensor([-3.5619, -4.0951, -3.6673, -2.8357, -4.7341,  2.1568, -2.5343,  4.4799,
         6.1202,  4.7775])
TensorRT: [-3.563822  -4.0938277 -3.6695344 -2.8367448 -4.7342696  2.1580381
 -2.529145   4.472117   6.1067324  4.7731795]


In [7]:
tensorrt_output

array([[-3.56468844e+00, -4.09343672e+00, -3.67025805e+00,
        -2.83583307e+00, -4.73365116e+00,  2.15937638e+00,
        -2.52551246e+00,  4.47502899e+00,  6.11003780e+00,
         4.77815390e+00,  3.13727617e+00,  5.90626001e+00,
        -3.33273560e-01,  3.58156228e+00,  2.00163436e+00,
         6.17331123e+00,  4.71036911e+00,  5.22766781e+00,
         1.64498940e+01,  1.55936766e+00,  8.93960476e+00,
         9.11338329e+00,  5.82359076e+00,  9.26880360e+00,
         3.86119056e+00,  3.98992300e+00,  2.55849862e+00,
        -3.25195253e-01,  1.13354301e+00,  1.71742201e+00,
        -3.13251436e-01, -2.18315482e+00, -9.31978941e-01,
        -5.53848791e+00, -2.55483007e+00, -2.85994315e+00,
        -2.33441067e+00, -2.93612838e+00,  2.56323874e-01,
        -1.82660627e+00, -1.44587386e+00,  8.03206503e-01,
        -2.17043400e+00, -2.22206259e+00, -1.19699550e+00,
        -6.93717718e-01, -2.35134435e+00, -2.15511298e+00,
         3.14973742e-01, -7.32031882e-01, -1.48200405e+0